# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a Data Frame that loads the csv file exported in Part One
my_cities_df = pd.read_csv("Output/City_Data_Farenheit.csv", encoding="utf-8")

# Print the number of rows in the DataFrame (just to check that it matches the rows exported to the csv file in Part One)
print(len(my_cities_df))

# Display the first 5 rows of the DataFrame
my_cities_df.head()


540


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nicoya,20,CR,1583973964,58,10.15,-85.45,84.20,18.34
1,Port Elizabeth,90,ZA,1583973904,88,-33.92,25.57,68.00,11.41
2,Leningradskiy,63,RU,1583973441,78,69.38,178.42,-8.52,12.17
3,Fortuna,1,US,1583974003,76,40.60,-124.16,57.00,14.99
4,Hirado,10,JP,1583974005,57,33.36,129.55,51.80,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a DataFrame called locations
locations = my_cities_df[["Lat", "Lng"]]

# Create a DataFrame called humidity
humidity = my_cities_df["Humidity"].astype(float)

# Create a variable called for the max humidity
max_humidity = humidity.max()

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create a Data Frame for cities with good weather based on our criteria
my_good_weather_cities_df = my_cities_df.loc[(my_cities_df["Max Temp"] > 70) & (my_cities_df["Max Temp"] < 80) & (my_cities_df["Cloudiness"] == 0) & (my_cities_df["Wind Speed"] < 10), :]

# Drop any rows that do not meet all of our criteria for good weather
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Reset the index for the Data Frame
my_good_weather_cities_df.reset_index(inplace=True)

# Delete the old Index
del my_good_weather_cities_df['index']

# Check the number of rows in the DataFrame
print(len(my_good_weather_cities_df))

# Dispaly the top 5 rows of the DataFrame
my_good_weather_cities_df.head()


12


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Itararé,0,BR,1583974115,91,-24.11,-49.33,72.00,5.99
1,Inhambane,0,MZ,1583974212,76,-23.86,35.38,77.40,6.44
2,Quang Ngai,0,VN,1583974223,79,15.12,108.80,74.39,4.85
3,Beloha,0,MG,1583974410,92,-25.17,45.05,70.30,6.87
4,Phayuha Khiri,0,TH,1583974447,68,15.46,100.14,79.03,7.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Iterate through the DataFrame and get the hotel name within the specified parameters for the cities with good weather

for index, row in my_good_weather_cities_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        # "type": "hotel",
        "keyword": "hotel",
        "key": g_key
    }
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json = response.json()
  
    try:
        my_good_weather_cities_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
# Drop the rows which do not match all the criteria
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Dispaly the top 5 rows of the DataFrame
my_good_weather_cities_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Itararé,0,BR,1583974115,91,-24.11,-49.33,72.00,5.99,Hospedaria Frei Caneca
1,Inhambane,0,MZ,1583974212,76,-23.86,35.38,77.40,6.44,Hotel Casa Do Capitao
2,Quang Ngai,0,VN,1583974223,79,15.12,108.80,74.39,4.85,Thien An Riverside Hotel
4,Phayuha Khiri,0,TH,1583974447,68,15.46,100.14,79.03,7.54,Chao Phraya Home
5,Kānkon,0,IN,1583974464,83,15.02,74.02,73.40,4.70,The LaLiT Golf & Spa Resort Goa


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in my_good_weather_cities_df.iterrows()]
locations = my_good_weather_cities_df[["Lat", "Lng"]]


In [8]:
# Add marker layer on top of heat map
hotel_name = my_good_weather_cities_df["Hotel Name"].tolist()
hotel_city = my_good_weather_cities_df["City"].tolist()
hotel_info = [hotel_name, hotel_city]
locations = my_good_weather_cities_df[["Lat", "Lng"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=[f"Name: {name}" for name in hotel_info[0]])

# Add the layer to the map
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))